In [ ]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.models import model_from_json

In [ ]:
features=[]
target=[]
for x in ['without_mask', 'with_mask']:
    ImagesNamesList=os.listdir("data" + "/" + str(x) )
    for y in ImagesNamesList:
        Imgarr=cv2.imread("data"+ "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(100,100))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="without_mask":
                target.append(0)
            else:
                target.append(1)
        
    #print("In Folder", x)

In [ ]:
features=np.array(features)

In [ ]:
target=np.array(target)

In [ ]:
features.shape

In [ ]:
target.shape

In [ ]:
features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.2)

In [ ]:
features_train.shape

In [ ]:
target_train.shape

In [ ]:
features_test.shape

In [ ]:
target_test.shape

In [ ]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255
    return image

In [ ]:
features_train=np.array(list(map(preprocessing,features_train)))

In [ ]:
features_train=features_train.reshape(6042, 100, 100,1)

In [ ]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)

In [ ]:
dataGen.fit(features_train)

In [ ]:
batches=dataGen.flow(features_train,target_train,batch_size=20)

In [ ]:
images,labels=next(batches)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(0,20):
    plt.subplot(4,5,i+1)
    plt.imshow(images[i].reshape(100,100))
plt.show()
    

In [ ]:
target_train=to_categorical(target_train)

In [ ]:
target_train.shape

# Step 1:

In [ ]:
model=Sequential()
model.add(Conv2D(100,(3,3),activation="relu",input_shape=(100,100,1)))
model.add(Conv2D(200,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))

# Step 2:

In [ ]:
model.compile(Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

# Step 3:

In [ ]:
model.fit(dataGen.flow(features_train,target_train,batch_size=20),epochs=20)

# Step 4:

In [ ]:
from keras.models import model_from_json

In [ ]:
model_json=model.to_json()
with open("AICorona.json","w") as abc:
    abc.write(model_json)
    abc.close()
model.save_weights("AICoronaWeights.h5")
print("Save the Model")

In [ ]:
json_file=open("AICorona.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("AICoronaWeights.h5")
print("Loaded Model Sucesssfully")

In [ ]:
import numpy as np
import cv2

 

def getClassName(classNo):
    if   classNo == 0: return 'Without Mask'
    elif classNo == 1: return 'Mask'
   


In [ ]:
capt=cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,180)

In [ ]:
while True:
    message,image=capt.read()
    imagearr=np.asarray(image)
    face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.05,3)
    for (x,y,w,h) in faces:
        detectedFace=image[x:x+w,y:y+h]
        detectedFace=cv2.resize(detectedFace,(100,100))
        detectedFace=preprocessing(detectedFace)
        detectedFace=detectedFace.reshape(1,100,100,1)
        predictions=loaded_model.predict(detectedFace)
        classIndex=loaded_model.predict_classes(imagearr)
        probabilityValue=np.amax(predictions)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        if probabilityValue>0.75:
            cv2.putText(image,getClassName(classIndex),(x , y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.putText(image,str(int(probabilityValue * 100)) + " %",( x+w+20,y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        cv2.imshow("Model Prediction",image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        
        
    
    
    
    
    
        
    
    
    
    
    

\ mean the character that is going to come after it, do not consider it as a part of the syntax. Instead consider it as a character only